# Этап 1

In [1]:
import cudf.pandas  # Импортирование cuDF и активация его использования
cudf.pandas.install()  # Установка cuDF как основного интерфейса для pandas
import os
import pandas as pd
from tqdm import tqdm
import torch
import pyarrow.parquet as pq
import dask.dataframe as dd
import spacy
import re
import pandas as pd
from datasets import Dataset as hfDataset
from transformers import AutoTokenizer, AutoModel, BertTokenizerFast
import torch
from sklearn.cluster import DBSCAN
import numpy as np
from collections import Counter
import nltk
from nltk.corpus import stopwords
# Проверка доступности GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN
import torch
from transformers import BertTokenizerFast, BertForSequenceClassification, BertConfig
import hdbscan
from scipy.spatial.distance import pdist, squareform
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader

In [2]:
result_limited = pd.read_csv("./reviews_keywords/wildberries_reviews.csv", nrows=1000)

In [3]:

# Загрузка дообученной модели и токенизатора
# Загружаем конфигурацию модели


# Загрузка модели и токенайзера от Сбербанка
tokenizer = BertTokenizerFast.from_pretrained("./reviews_keywords/fine_tuned_model")
model = AutoModel.from_pretrained("./reviews_keywords/fine_tuned_model").to(device)
# Инициализируем модель с конфигурацией
config = BertConfig.from_pretrained('./reviews_keywords/fine_tuned_model', output_hidden_states=True)
model_classification = BertForSequenceClassification.from_pretrained('./reviews_keywords/fine_tuned_model', config=config).to(device)

spacy.prefer_gpu()
# Загрузка и настройка модели SpaCy
nlp = spacy.load("ru_core_news_lg", disable=["ner", "tagger", "attribute_ruler", "lemmatizer"])

df = result_limited

# Преобразование pandas DataFrame в Hugging Face Dataset
dataset = hfDataset.from_pandas(df)

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
spacy.prefer_gpu()
# Загрузка и настройка модели SpaCy
nlp = spacy.load("ru_core_news_lg", disable=["ner", "tagger", "attribute_ruler", "lemmatizer"])


In [5]:
def clean_text(text):
    text = re.sub(r'[\n\r\t]+|\s{2,}', ' ', text)  # Объединяем шаги для замены пробелов
    text = re.sub(r'(?<!\.)\s*\.\s*|\s*\.\s*(?!\.)', '. ', text)  # Оптимизация замены точки
    return text.strip().rstrip('.')

def split_by_syntax(doc, max_length=100):
    """
    Разделяет длинные предложения на более короткие, основываясь на синтаксическом анализе.
    """
    split_sentences = []
    current_chunk = []
    if len(doc.text) < 100:
        return [doc.text]
    for token in doc:
        current_chunk.append(token.text)
        
        # Проверка, можно ли завершить предложение
        if token.dep_ in ('punct', 'cc', 'conj') or len(' '.join(current_chunk)) > max_length:
            if len(' '.join(current_chunk)) > max_length:
                split_sentences.append(' '.join(current_chunk[:-1]))
                current_chunk = [current_chunk[-1]]
            else:
                split_sentences.append(' '.join(current_chunk))
                current_chunk = []

    # Добавление последнего оставшегося фрагмента
    if current_chunk:
        split_sentences.append(' '.join(current_chunk))

    return split_sentences

def split_reviews_into_sentences(batch):
    # Очистка текстов
    cleaned_texts = [clean_text(text) for text in batch['corrected_text']]
    
    # Обработка текстов с помощью nlp.pipe с указанием batch_size
    docs = list(nlp.pipe(cleaned_texts, batch_size=64))

    # Извлечение предложений и их дальнейшая обработка
    processed_sentences = []
    for doc in docs:
        sentences = []
        for sent in doc.sents:
            # Разделяем длинные предложения по синтаксису
            split_sentences = split_by_syntax(sent, max_length=100)
            sentences.extend(split_sentences)
        processed_sentences.append(sentences)
    
    batch['sentences'] = processed_sentences
    
    return batch

dataset = dataset.map(split_reviews_into_sentences, batched=True, batch_size=32)

# Преобразуем Dataset обратно в pandas DataFrame
df = dataset.to_pandas()

# Выполним explode по столбцу с предложениями
df_exploded = df.explode('sentences').reset_index(drop=True)

# Удаляем лишние столбцы, которые появились после explode
df_exploded = df_exploded.drop(columns=[col for col in df_exploded.columns if col.startswith('__index_level_')])

# Преобразуем DataFrame обратно в Hugging Face Dataset
dataset_exploded = hfDataset.from_pandas(df_exploded)

from torch.cuda.amp import autocast

def compute_sentence_embeddings(sentences):
    # Фильтруем список, оставляя только строки
    sentences = [str(sentence) for sentence in sentences if isinstance(sentence, str)]
    
    if not sentences:
        raise ValueError("Input contains no valid strings.")

    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt").to(device)
    
    with torch.no_grad():
        with autocast():  # Используем mixed precision для ускорения
            outputs = model(**inputs)
    
    return outputs.last_hidden_state.mean(dim=1).cpu().numpy()

def compute_embeddings(batch, imput_col = 'sentences', output_col = 'sentence_embeddings'):
    sentences = batch[imput_col]

    # Проверяем, что все элементы в batch являются строками
    valid_sentences = [str(sentence) for sentence in sentences if isinstance(sentence, str)]
    
    if not valid_sentences:
        batch[output_col] = [[]] * len(sentences)  # Если нет валидных предложений, возвращаем пустые эмбеддинги
        return batch

    embeddings = compute_sentence_embeddings(valid_sentences)

    # Приведение эмбеддингов к типу float32 для консистентности
    embeddings = embeddings.astype(np.float32)

    # Проверяем, что количество эмбеддингов совпадает с количеством предложений
    if len(embeddings) != len(valid_sentences):
        raise ValueError("Количество эмбеддингов не совпадает с количеством предложений.")
    
    # Если количество предложений после фильтрации не совпадает с исходным, корректируем выходные данные
    final_embeddings = []
    embed_idx = 0
    for sentence in sentences:
        if isinstance(sentence, str):
            final_embeddings.append(embeddings[embed_idx])
            embed_idx += 1
        else:
            final_embeddings.append(np.zeros(embeddings.shape[1], dtype=np.float32))  # Добавляем нулевые эмбеддинги для невалидных предложений

    batch[output_col] = final_embeddings
    return batch

def compute_embeddings_after_explode(batch, imput_col = 'sentences', output_col = 'sentence_embeddings'):
    return compute_embeddings(batch, imput_col = imput_col, output_col = output_col)

# Применение функции
dataset = dataset_exploded.map(compute_embeddings_after_explode, batched=True, batch_size=128)
df_embed = dataset.to_pandas()


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3243 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/tmp/ipykernel_2052/1857095033.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Используем mixed precision для ускорения


In [6]:
df_embed.head()

,Unnamed: 0,review_full_text,review_rating,product,category,url,corrected_text,sentences,__index_level_0__,sentence_embeddings
0,0,Работает хорошо.,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Работает хорошо.,Работает хорошо,0,"[0.1565534770488739, -0.6669323444366455, -1.0..."
1,1,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Пришло быстро, все целое на вид. Завтра буду и...","Пришло быстро, все целое на вид.",1,"[0.014507248997688293, -0.5217506289482117, -0..."
2,1,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Пришло быстро, все целое на вид. Завтра буду и...",Завтра буду испытывать,2,"[0.08877117931842804, 0.21360503137111664, 1.3..."
3,2,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Купил на квадр для поднятия отвала, установка ...","Купил на квадр для поднятия отвала, установка ...",3,"[-0.1982501894235611, -0.017847895622253418, 0..."
4,3,Лебёдка хорошая. Но в инструкции ни слова про ...,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Лебёдка хорошая. Но в инструкции ни слова про ...,Лебёдка хорошая.,4,"[-0.5404854416847229, -0.10484828799962997, 0...."


In [13]:


# Определение устройства (GPU или CPU)
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# Перевод модели в режим FP16, если это возможно
if use_cuda:
    model_classification = model_classification.half()

# Пример данных (замените на реальные данные)
reviews = df_embed["sentences"]

# Очистка данных от некорректных значений
reviews = [str(review) for review in reviews if isinstance(review, str) and review.strip()]

# Создание кастомного Dataset для обработки отзывов
class ReviewDataset(Dataset):
    def __init__(self, reviews, tokenizer, max_len=128):
        self.reviews = reviews
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, idx):
        review = self.reviews[idx]
        encoding = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {key: val.flatten() for key, val in encoding.items()}

# Создаем датасет и DataLoader
dataset = ReviewDataset(reviews, tokenizer)
batch_size = 32  # Размер батча можно изменить в зависимости от объема доступной памяти GPU
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# Получение предсказаний с отображением прогресса
predictions = []

from torch.cuda.amp import autocast  # Импортируем autocast для смешанной точности

for batch in tqdm(dataloader, desc="Предсказание отзывов"):
    batch = {key: val.to(device) for key, val in batch.items()}
    
    with torch.no_grad():
        with autocast():  # Используем смешанную точность
            outputs = model_classification(**batch)
            logits = outputs[0] if isinstance(outputs, tuple) else outputs.logits
            probabilities = torch.softmax(logits, dim=-1)
            batch_predictions = (probabilities[:, 1] > 0.7).cpu().numpy()  # Используем порог 0.7
            predictions.extend(batch_predictions)

# Преобразование в DataFrame, если это еще не сделано
if not isinstance(df_embed, pd.DataFrame):
    df_embed = pd.DataFrame(df_embed)

# Проверка и обработка несоответствия длины
if len(predictions) != len(df_embed):
    print(f"Warning: Length of predictions ({len(predictions)}) does not match length of index ({len(df_embed)})")
    
    # Пример: Заполнение недостающих значений
    if len(predictions) < len(df_embed):
        missing_count = len(df_embed) - len(predictions)
        predictions.extend([0] * missing_count)  # Добавляем нули в случае недостатка предсказаний

    elif len(predictions) > len(df_embed):
        predictions = predictions[:len(df_embed)]  # Обрезаем список предсказаний

# Присоединение предсказаний к датасету
df_embed['predictions'] = predictions
df_embed.head()

Предсказание отзывов: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 102/102 [00:16<00:00,  6.06it/s]


,Unnamed: 0,review_full_text,review_rating,product,category,url,corrected_text,sentences,__index_level_0__,sentence_embeddings,predictions
0,0,Работает хорошо.,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Работает хорошо.,Работает хорошо,0,"[0.1565534770488739, -0.6669323444366455, -1.0...",False
1,1,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Пришло быстро, все целое на вид. Завтра буду и...","Пришло быстро, все целое на вид.",1,"[0.014507248997688293, -0.5217506289482117, -0...",False
2,1,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Пришло быстро, все целое на вид. Завтра буду и...",Завтра буду испытывать,2,"[0.08877117931842804, 0.21360503137111664, 1.3...",True
3,2,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Купил на квадр для поднятия отвала, установка ...","Купил на квадр для поднятия отвала, установка ...",3,"[-0.1982501894235611, -0.017847895622253418, 0...",True
4,3,Лебёдка хорошая. Но в инструкции ни слова про ...,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Лебёдка хорошая. Но в инструкции ни слова про ...,Лебёдка хорошая.,4,"[-0.5404854416847229, -0.10484828799962997, 0....",True


In [14]:
# Группировка и объединение предложений в одной строке для каждого уникального значения "corrected_text"
grouped_sentences = df_embed[df_embed["predictions"] == 1].groupby("corrected_text")['sentences'].apply(lambda x: " ".join(x)).reset_index()

# Переименовываем столбец для удобства
grouped_sentences = grouped_sentences.rename(columns={'sentences': 'combined_sentences'})

# Выводим результат
grouped_sentences

/opt/conda/lib/python3.10/site-packages/cudf/core/groupby/groupby.py:1343: RuntimeWarning: GroupBy.apply() performance scales poorly with number of groups. Got 523 groups. Some functions may perform better by passing engine='jit'
  warnings.warn(


,corrected_text,combined_sentences
0,)))из минуса только то что на сумке нет собачк...,из минуса только то что на сумке нет собачки н...
1,"2 раза спасали на гололедице, шлифовал на мест...","2 раза спасали на гололедице, шлифовал на мест..."
2,3из пяти помогли. Узковаты,3из пяти помогли. Узковаты
3,Авто парфюм реально суперский🔥все ришло в цел...,Авто парфюм реально суперский 🔥 все ришло в це...
4,"Анти буксы - это вещь! В снегопады спасают, чт...","Анти буксы - это вещь! В снегопады спасают, чт..."
...,...,...
518,Я не ожидал такого комплекта с арками саморезы...,Я не ожидал такого комплекта с арками саморезы...
519,"Я очень доволен ! \nВыглядят массивно, крепко....","Выглядят массивно, крепко. Сумку для инструмен..."
520,"Я просто в восторге ! Товар пришел быстро, зап...",сама уже хочу
521,"спасут, но не везде","спасут, но не везде"


In [15]:
import re
def has_multiple_words(sentence):
    # Убираем пунктуацию и разделяем предложение по пробелам
    words = re.findall(r'\b\w+\b', sentence)
    return len(words) > 1
df_filtered_reviews = df_embed[df_embed["predictions"] == 1].drop(["Unnamed: 0", "__index_level_0__", "predictions"], axis=1)
# df_filtered_reviews["corrected_filtered_text"] = grouped_sentences['combined_sentences']
df_filtered_reviews = df_filtered_reviews[df_filtered_reviews['sentences'].apply(has_multiple_words)]
df_filtered_reviews = pd.merge(df_filtered_reviews, grouped_sentences, on='corrected_text', how='left')

# Группировка по 'corrected_text' и объединение предложений в 'sentences'
# df_filtered_reviews = df_embed[df_embed["predictions"] == 1].groupby(["review_full_text", "review_rating", "product", "category", "url", "corrected_text"])['sentences'].apply(lambda x: " ".join(x)).reset_index()
df_filtered_reviews = df_embed[df_embed["predictions"] == 1].drop(["Unnamed: 0", "__index_level_0__", "predictions"], axis=1)
# df_filtered_reviews["corrected_filtered_text"] = grouped_sentences['combined_sentences']
df_filtered_reviews = df_filtered_reviews[df_filtered_reviews['sentences'].apply(has_multiple_words)]
df_filtered_reviews = pd.merge(df_filtered_reviews, grouped_sentences, on='corrected_text', how='left')
df_filtered_reviews.head()

,review_full_text,review_rating,product,category,url,corrected_text,sentences,sentence_embeddings,combined_sentences
0,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Пришло быстро, все целое на вид. Завтра буду и...",Завтра буду испытывать,"[0.08877117931842804, 0.21360503137111664, 1.3...",Завтра буду испытывать
1,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Купил на квадр для поднятия отвала, установка ...","Купил на квадр для поднятия отвала, установка ...","[-0.1982501894235611, -0.017847895622253418, 0...","Купил на квадр для поднятия отвала, установка ..."
2,Лебёдка хорошая. Но в инструкции ни слова про ...,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Лебёдка хорошая. Но в инструкции ни слова про ...,Лебёдка хорошая.,"[-0.5404854416847229, -0.10484828799962997, 0....",Лебёдка хорошая. Но в инструкции ни слова про ...
3,Лебёдка хорошая. Но в инструкции ни слова про ...,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Лебёдка хорошая. Но в инструкции ни слова про ...,Но в инструкции ни слова про сборку и креплени...,"[0.2806070148944855, 0.5317667126655579, 0.896...",Лебёдка хорошая. Но в инструкции ни слова про ...
4,Муж ещё не пробовал,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Муж ещё не пробовал,Муж ещё не пробовал,"[0.2862488627433777, 0.46963462233543396, 0.82...",Муж ещё не пробовал


In [16]:
def compute_embeddings_after_filter(batch, imput_col = 'combined_sentences', output_col = 'combined_sentences_embeddings'):
    return compute_embeddings(batch, imput_col = imput_col, output_col = output_col)

dataset_filtered = hfDataset.from_pandas(df_filtered_reviews)

df_filtered_embed = dataset_filtered.map(compute_embeddings_after_filter, batched=True, batch_size=128).to_pandas().drop("__index_level_0__",axis=1)

Map:   0%|          | 0/1373 [00:00<?, ? examples/s]

In [17]:
# Загрузка модели spaCy для русского языка
nlp = spacy.load("ru_core_news_lg")

# Установка стоп-слов
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# combined_sentences

In [18]:
df_filtered_embed

,review_full_text,review_rating,product,category,url,corrected_text,sentences,sentence_embeddings,combined_sentences,combined_sentences_embeddings
0,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Пришло быстро, все целое на вид. Завтра буду и...",Завтра буду испытывать,"[0.08877117931842804, 0.21360503137111664, 1.3...",Завтра буду испытывать,"[0.11882603913545609, 0.22687514126300812, 1.3..."
1,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Купил на квадр для поднятия отвала, установка ...","Купил на квадр для поднятия отвала, установка ...","[-0.1982501894235611, -0.017847895622253418, 0...","Купил на квадр для поднятия отвала, установка ...","[-0.2166317254304886, -0.07144202291965485, 0...."
2,Лебёдка хорошая. Но в инструкции ни слова про ...,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Лебёдка хорошая. Но в инструкции ни слова про ...,Лебёдка хорошая.,"[-0.5404854416847229, -0.10484828799962997, 0....",Лебёдка хорошая. Но в инструкции ни слова про ...,"[0.35663720965385437, 0.46731141209602356, 0.8..."
3,Лебёдка хорошая. Но в инструкции ни слова про ...,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Лебёдка хорошая. Но в инструкции ни слова про ...,Но в инструкции ни слова про сборку и креплени...,"[0.2806070148944855, 0.5317667126655579, 0.896...",Лебёдка хорошая. Но в инструкции ни слова про ...,"[0.35663720965385437, 0.46731141209602356, 0.8..."
4,Муж ещё не пробовал,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Муж ещё не пробовал,Муж ещё не пробовал,"[0.2862488627433777, 0.46963462233543396, 0.82...",Муж ещё не пробовал,"[0.27887243032455444, 0.4692499041557312, 0.83..."
...,...,...,...,...,...,...,...,...,...,...
1368,Средство рабочее надо брать,5,ПолиКомПласт / Преобразователь очиститель ржав...,/Автотовары/Автокосметика и автохимия,https://www.wildberries.ru/catalog/155565431/f...,Средство рабочее надо брать,Средство рабочее надо брать,"[0.023214329034090042, 0.06771326810121536, 0....",Средство рабочее надо брать,"[0.03595263510942459, 0.08840777724981308, 0.3..."
1369,Хорошо убирает ржавчину,5,ПолиКомПласт / Преобразователь очиститель ржав...,/Автотовары/Автокосметика и автохимия,https://www.wildberries.ru/catalog/155565431/f...,Хорошо убирает ржавчину,Хорошо убирает ржавчину,"[0.007927305065095425, -0.016519170254468918, ...",Хорошо убирает ржавчину,"[0.12215739488601685, 0.006324803456664085, 0...."
1370,"так не о чем, эффекта почти нет, бестолковая х...",3,ПолиКомПласт / Преобразователь очиститель ржав...,/Автотовары/Автокосметика и автохимия,https://www.wildberries.ru/catalog/155565431/f...,"Так не о чем, эффекта почти нет, бестолковая х...","Так не о чем, эффекта почти нет, бестолковая х...","[0.7261900305747986, 1.162724494934082, -0.363...","Так не о чем, эффекта почти нет, бестолковая х...","[0.7124964594841003, 1.157876968383789, -0.309..."
1371,Посмотрим что и как будет,5,ПолиКомПласт / Преобразователь очиститель ржав...,/Автотовары/Автокосметика и автохимия,https://www.wildberries.ru/catalog/155565431/f...,Посмотрим что и как будет,Посмотрим что и как будет,"[-1.018591284751892, 0.10584204643964767, 1.29...",Посмотрим что и как будет,"[-0.9789733290672302, 0.12944084405899048, 1.3..."


In [19]:
def compute_embeddings_after_filter(batch, imput_col = 'combined_sentences', output_col = 'combined_sentences_embeddings'):
    return compute_embeddings(batch, imput_col = imput_col, output_col = output_col)

dataset_filtered = hfDataset.from_pandas(df_filtered_reviews)

df_filtered_embed = dataset_filtered.map(compute_embeddings_after_filter, batched=True, batch_size=128).to_pandas().drop("__index_level_0__",axis=1)

# Удаляем дубликаты на основе ключевых столбцов
df_filtered_embed_grouped = df_filtered_embed.drop_duplicates(
    subset=['review_full_text', 'review_rating', 'product', 'category', 'url', 'corrected_text', 'combined_sentences', 'combined_sentences_embeddings']
)

# Агрегируем предложения и эмбеддинги в списки
df_filtered_embed_grouped = df_filtered_embed_grouped.groupby(
    ['review_full_text', 'review_rating', 'product', 'category', 'url', 'corrected_text', 'combined_sentences', 'combined_sentences_embeddings']
).agg({
    'sentences': list,  # Агрегируем предложения в список
    'sentence_embeddings': list,  # Агрегируем эмбеддинги предложений в список
}).reset_index()

df_filtered_embed_grouped


Map:   0%|          | 0/1373 [00:00<?, ? examples/s]

,review_full_text,review_rating,product,category,url,corrected_text,combined_sentences,combined_sentences_embeddings,sentences,sentence_embeddings
0,)))из минуса только то что на сумке нет собачк...,5,ВПМ / Антибукс - антипробуксовочные траки утол...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/48839109/fe...,)))из минуса только то что на сумке нет собачк...,из минуса только то что на сумке нет собачки н...,"[-0.13375142216682434, 0.40174728631973267, 0....",[из минуса],"[[0.09002872556447983, 0.005189549643546343, -..."
1,"2 раза спасали на гололедице,шлифовал на месте...",5,ВПМ / Антибукс - антипробуксовочные траки утол...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/48839109/fe...,"2 раза спасали на гололедице, шлифовал на мест...","2 раза спасали на гололедице, шлифовал на мест...","[0.15654981136322021, 0.35932230949401855, -0....","[2 раза спасали на гололедице, шлифовал на мес...","[[0.14486153423786163, 0.3898766040802002, -0...."
2,3из пяти помогли. Узковаты,4,ВЫРУЧАЙКА / Антибукс Противобуксовочные траки ...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/62827233/fe...,3из пяти помогли. Узковаты,3из пяти помогли. Узковаты,"[-0.07311840355396271, 0.23892344534397125, -0...",[3из пяти помогли.],"[[-0.9125573635101318, -0.08403690159320831, -..."
3,Автопарфюм реально суперский🔥все пришло в цел...,5,Пахнет и Точка / Ароматизатор в машину автопар...,/Автотовары/Автокосметика и автохимия,https://www.wildberries.ru/catalog/197410221/f...,Авто парфюм реально суперский🔥все ришло в цел...,Авто парфюм реально суперский 🔥 все ришло в це...,"[-0.6257163286209106, 0.07905878871679306, -0....",[Авто парфюм реально суперский 🔥 все ришло в ц...,"[[-0.6403707265853882, 0.12857548892498016, -0..."
4,"Антибуксы - это вещь! В снегопады спасают, что...",5,ВПМ / Антибукс - антипробуксовочные траки утол...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/48839109/fe...,"Анти буксы - это вещь! В снегопады спасают, чт...","Анти буксы - это вещь! В снегопады спасают, чт...","[0.010491854511201382, 0.34159913659095764, 0....",[Анти буксы - это вещь!],"[[-0.2598613202571869, -0.22514109313488007, 0..."
...,...,...,...,...,...,...,...,...,...,...
590,спасут\nно не везде,5,ВЫРУЧАЙКА / Антибукс Противобуксовочные траки ...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/62827233/fe...,"спасут, но не везде","спасут, но не везде","[-0.5613567233085632, 0.39890602231025696, 0.7...","[спасут, но не везде]","[[-0.5418501496315002, 0.40518274903297424, 0...."
591,"суперштучки) для женщины самое то - не копать,...",5,ВЫРУЧАЙКА / Антибукс Противобуксовочные траки ...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/62827233/fe...,Супер штучки) для женщины самое то - ни копать...,"для женщины самое то - ни копать, ни ждать, по...","[0.07101850211620331, 0.2978771924972534, 0.37...","[для женщины самое то - ни копать, ни ждать, п...","[[0.1675279587507248, 0.4484736919403076, 0.25..."
592,"так не о чем, эффекта почти нет, бестолковая х...",3,ПолиКомПласт / Преобразователь очиститель ржав...,/Автотовары/Автокосметика и автохимия,https://www.wildberries.ru/catalog/155565431/f...,"Так не о чем, эффекта почти нет, бестолковая х...","Так не о чем, эффекта почти нет, бестолковая х...","[0.7124964594841003, 1.157876968383789, -0.309...","[Так не о чем, эффекта почти нет, бестолковая ...","[[0.7261900305747986, 1.162724494934082, -0.36..."
593,установили. сразу начала подозрительно хрустет...,2,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Установили. Сразу начала подозрительно хрустет...,"Второй раз включил ее попробовать, всё, уже не...","[0.36054134368896484, 0.5008261799812317, 0.42...","[Второй раз включил ее попробовать, всё, уже н...","[[0.41353243589401245, 0.41926252841949463, 0...."


In [20]:
import numpy as np
import pandas as pd
from hdbscan import HDBSCAN
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_distances
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist
from tqdm import tqdm
from loguru import logger

logger.add('./reviews_keywords/clustering.log', encoding='utf-8')
# Перевод модели в режим FP16, если это возможно
if torch.cuda.is_available():
    model_classification = model_classification.half()

def find_centroid(embeddings):
    return np.mean(embeddings, axis=0)

def clean_text(text):
    """Функция для удаления разделителей из текста"""
    return re.sub(r'[\n\t\r]', ' ', text).strip()

def count_words(cluster_sentences):
    words = " ".join(cluster_sentences).split()
    return len(words)

def compute_mean_distance(cluster_indices, embeddings, metric='cosine'):
    if len(cluster_indices) < 2:
        return 0
    cluster_embeddings = embeddings[cluster_indices]
    pairwise_distances = pdist(cluster_embeddings, metric=metric)
    return np.mean(pairwise_distances)

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import re

def extract_key_thought(sentences, sentence_embeddings, review_full_texts, combined_sentences_embeddings, top_n=2):
    # Преобразуем списки эмбеддингов в массивы NumPy
    sentence_embeddings = [np.array(embed) for embed in sentence_embeddings]
    combined_sentences_embeddings = [np.array(embed) for embed in combined_sentences_embeddings]
    
    # Вычисляем центроид для предложений
    sentence_centroid = find_centroid(np.vstack(sentence_embeddings))
    
    # Вычисляем косинусное сходство для предложений
    sentence_similarities = [
        cosine_similarity(embed.reshape(1, -1), sentence_centroid.reshape(1, -1)).flatten()[0] 
        for embed in sentence_embeddings
    ]
    
    # Выбираем топ N предложений
    top_sentence_indices = np.argsort(sentence_similarities)[-top_n:][::-1]
    top_sentences = [sentences[i] for i in top_sentence_indices]
    
    # Вычисляем косинусное сходство для отзывов
    review_similarities = [
        cosine_similarity(embed.reshape(1, -1), sentence_centroid.reshape(1, -1)).flatten()[0] 
        for embed in combined_sentences_embeddings
    ]
    
    # Выбираем топ N отзывов и очищаем их от лишних символов
    top_review_indices = np.argsort(review_similarities)[-top_n:][::-1]
    top_reviews = [clean_text(review_full_texts[i]) for i in top_review_indices]
    
    # Объединяем топ предложения и отзывы в один список
    key_thought = top_sentences + top_reviews
    
    return key_thought

def extract_key_thought_with_sentiment(sentences, sentence_embeddings, review_full_texts, combined_sentences_embeddings, review_ratings, top_n=2):
    # Разделяем предложения и отзывы по их оценке
    positive_indices = [i for i, rating in enumerate(review_ratings) if rating > 3]
    negative_indices = [i for i, rating in enumerate(review_ratings) if rating <= 3]
    
    key_thoughts = {'positive': [], 'negative': []}
    
    # Обработка позитивных отзывов
    if positive_indices:
        positive_sentences = [sentences[i][0] for i in positive_indices]
        positive_sentence_embeddings = [sentence_embeddings[i] for i in positive_indices]
        positive_reviews = [review_full_texts[i] for i in positive_indices]
        positive_combined_embeddings = [combined_sentences_embeddings[i] for i in positive_indices]
        
        key_thoughts['positive'] = extract_key_thought(
            positive_sentences, 
            positive_sentence_embeddings, 
            positive_reviews, 
            positive_combined_embeddings, 
            top_n=top_n
        )
    
    # Обработка негативных отзывов
    if negative_indices:
        negative_sentences = [sentences[i][0] for i in negative_indices]
        negative_sentence_embeddings = [sentence_embeddings[i] for i in negative_indices]
        negative_reviews = [review_full_texts[i] for i in negative_indices]
        negative_combined_embeddings = [combined_sentences_embeddings[i] for i in negative_indices]
        
        key_thoughts['negative'] = extract_key_thought(
            negative_sentences, 
            negative_sentence_embeddings, 
            negative_reviews, 
            negative_combined_embeddings, 
            top_n=top_n
        )
    
    return key_thoughts


def aggregate_clusters(group, labels, combined_embeddings, top_n=2):
    group['final_cluster'] = labels
    group['mean_distance'] = group['final_cluster'].apply(
        lambda cluster_id: compute_mean_distance(
            np.where(labels == cluster_id)[0], combined_embeddings)
    )
    
    grouped_clusters = group.groupby('final_cluster').agg({
        'category': 'first',
        'product': 'first',
        'review_rating': list,  # Список оценок
        'url': 'first',
        'sentences': list,
        'sentence_embeddings': list,
        'review_full_text': list,
        'combined_sentences': list,
        'combined_sentences_embeddings': list,
        'mean_distance': 'first',
    }).reset_index(drop=True)
    
    grouped_clusters['elem_count'] = grouped_clusters['sentences'].apply(len)
    grouped_clusters['review_rating_mean'] = grouped_clusters['review_rating'].apply(np.mean)
    grouped_clusters['word_count'] = grouped_clusters['sentences'].apply(lambda x: sum([len(" ".join(s).split()) for s in x]))
    
    # Применяем обновленную функцию для извлечения ключевых мыслей
    grouped_clusters['key_thoughts'] = grouped_clusters.apply(
        lambda row: extract_key_thought_with_sentiment(
            row['sentences'], 
            row['sentence_embeddings'], 
            row['review_full_text'], 
            row['combined_sentences_embeddings'], 
            row['review_rating'],
            top_n=top_n
        ), axis=1)
    
    return grouped_clusters.sort_values(by='word_count', ascending=False)



def cluster_with_hdbscan(group, min_cluster_size=3, min_samples=3):
    combined_embeddings = np.vstack(group["combined_sentences_embeddings"].values)
    distance_matrix = cosine_distances(combined_embeddings)
    
    hdb = HDBSCAN(metric='precomputed', min_cluster_size=min_cluster_size, min_samples=min_samples)
    labels = hdb.fit_predict(distance_matrix)
    
    return aggregate_clusters(group, labels, combined_embeddings)

def cluster_with_hierarchical(group, max_distance=0.5):
    combined_embeddings = np.array(list(group["combined_sentences_embeddings"]))
    distance_matrix = pdist(combined_embeddings, metric='cosine')
    linkage_matrix = linkage(distance_matrix, method='ward')
    labels = fcluster(linkage_matrix, t=max_distance, criterion='distance')
    
    return aggregate_clusters(group, labels, combined_embeddings)

def cluster_with_kmeans(group, n_clusters=5):
    combined_embeddings = np.array(list(group["combined_sentences_embeddings"]))
    n_samples = combined_embeddings.shape[0]
    
    if n_samples < n_clusters:
        n_clusters = max(2, n_samples)
    
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    labels = kmeans.fit_predict(combined_embeddings)
    
    return aggregate_clusters(group, labels, combined_embeddings)

def cluster_product_reviews(df, clustering_method, **kwargs):
    final_clusters_list = []
    grouped = df.groupby('product')

    for product, group in tqdm(grouped, desc="Processing products"):
        clustered_group = clustering_method(group, **kwargs)
        final_clusters_list.append(clustered_group)
    
    return pd.concat(final_clusters_list, ignore_index=True)


# Применение функций к вашему сгруппированному датафрейму
final_df_hdbscan = cluster_product_reviews(df_filtered_embed_grouped, cluster_with_hdbscan, min_cluster_size=3, min_samples=3)
final_df_hierarchical = cluster_product_reviews(df_filtered_embed_grouped, cluster_with_hierarchical, max_distance=0.3)
final_df_kmeans = cluster_product_reviews(df_filtered_embed_grouped, cluster_with_kmeans, n_clusters=20)

# Сортировка результатов для анализа
final_df_hdbscan = final_df_hdbscan.sort_values("word_count", ascending=False)
final_df_hierarchical = final_df_hierarchical.sort_values("word_count", ascending=False)
final_df_kmeans = final_df_kmeans.sort_values("word_count", ascending=False)

# Вывод результатов
display(f"hdbscan")
display(final_df_hdbscan[final_df_hdbscan["elem_count"] > 2].sort_values("product", ascending=False).head(5))
display(final_df_hdbscan[final_df_hdbscan["elem_count"] > 2].describe())
display(f"hierarchical")
display(final_df_hierarchical[final_df_hierarchical["elem_count"] > 2].sort_values("product", ascending=False).head(5))
display(final_df_hierarchical[final_df_hierarchical["elem_count"] > 2].describe())
display(f"kmeans")
display(final_df_kmeans[final_df_kmeans["elem_count"] > 2].sort_values("product", ascending=False).head(5))
display(final_df_kmeans[final_df_kmeans["elem_count"] > 2].describe())


Processing products: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:05<00:00,  2.83it/s]


'hdbscan'

,category,product,review_rating,url,sentences,sentence_embeddings,review_full_text,combined_sentences,combined_sentences_embeddings,mean_distance,elem_count,review_rating_mean,word_count,key_thoughts
17,/Автотовары/OFFroad,Фрегат Лифт Подвеска / Лифт комплект рессоры К...,"[5, 5, 3, 5]",https://www.wildberries.ru/catalog/111627554/f...,"[[Все встало без проблем, уаз поднялся на 3 см...","[[[-0.8197040557861328, -0.00407177209854126, ...","[Все встало без проблем, уаз поднялся на 3 см....","[Все встало без проблем, уаз поднялся на 3 см....","[[-0.8057578802108765, -0.10355747491121292, -...",0.491122,4,4.500000,38,"{'negative': ['На вид не плохие, но к сожалени..."
16,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,"[1, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",https://www.wildberries.ru/catalog/155565431/f...,"[[Не полная.], [Брала для бытовых нужд, очисти...","[[[-0.24720291793346405, 0.31582504510879517, ...","[Банка вскрыта.Не полная.Жаль,что сразу не пос...","[Не полная. Жаль, что сразу не посмотрел. Напи...","[[0.4564327299594879, 0.5242350101470947, 0.28...",0.411233,104,4.673077,703,{'negative': ['Совсем не убирает ржавчину с ма...
15,/Автотовары/Автокосметика и автохимия,Пахнет и Точка / Ароматизатор в машину автопар...,"[5, 5, 5, 5, 5, 5, 2, 5, 4, 5, 1, 1, 1, 3, 3, ...",https://www.wildberries.ru/catalog/197410221/f...,[[Авто парфюм реально суперский 🔥 все ришло в ...,"[[[-0.6403707265853882, 0.12857548892498016, -...",[Автопарфюм реально суперский🔥все пришло в це...,[Авто парфюм реально суперский 🔥 все ришло в ц...,"[[-0.6257163286209106, 0.07905878871679306, -0...",0.443262,69,4.028986,419,"{'negative': ['Запаха нет, но средство испорти..."
14,/Автотовары/OFFroad,ПК ЛИМ / Цепи противоскольжения для легковых а...,"[4, 5, 5, 5, 2, 5, 5, 5, 5, 3, 3, 4, 4, 3, 5, ...",https://www.wildberries.ru/catalog/143132161/f...,"[[Брал для поездки на определенное озеро ,], [...","[[[-0.4637119174003601, -0.0744853988289833, 0...","[Брал для поездки на определенное озеро, круто...","[Брал для поездки на определенное озеро , крут...","[[0.03720133379101753, 0.4053092896938324, 0.3...",0.346729,64,4.203125,464,"{'negative': ['Не удобные, застежка ремня сдел..."
13,/Автотовары/OFFroad,ПК ЛИМ / Браслеты цепи противоскольжения,"[4, 5, 5, 5, 2, 5, 5, 5, 5, 3, 3, 4, 4, 3, 5, ...",https://www.wildberries.ru/catalog/143132420/f...,"[[Брал для поездки на определенное озеро ,], [...","[[[-0.45377832651138306, -0.05294693633913994,...","[Брал для поездки на определенное озеро, круто...","[Брал для поездки на определенное озеро , крут...","[[0.03717208653688431, 0.4140254855155945, 0.3...",0.346440,64,4.203125,464,"{'negative': ['Не удобные, застежка ремня сдел..."


,mean_distance,elem_count,review_rating_mean,word_count
count,16.000000,16.000000,16.000000,16.000000
mean,0.379513,36.937500,4.392312,248.562500
std,0.142046,34.264595,0.353690,235.260131
min,0.135405,3.000000,3.941176,14.000000
25%,0.282215,11.750000,4.132246,75.250000
50%,0.390030,20.500000,4.372329,139.500000
75%,0.468604,64.000000,4.668269,430.250000
max,0.658028,104.000000,5.000000,717.000000


'hierarchical'

,category,product,review_rating,url,sentences,sentence_embeddings,review_full_text,combined_sentences,combined_sentences_embeddings,mean_distance,elem_count,review_rating_mean,word_count,key_thoughts
218,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,"[5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 3, 1, 5, ...",https://www.wildberries.ru/catalog/155565431/f...,"[[Не очищал вообще никак.], [не стекает], [Не ...","[[[0.12246482819318771, 0.38563504815101624, 0...","[В принципе, рабочая штука.\nНе очищал вообще ...",[Не очищал вообще никак. Огромный + то что кис...,"[[0.25050216913223267, 0.4493422508239746, -0....",0.117169,30,4.4,226,{'negative': ['Совсем не убирает ржавчину с ма...
219,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,"[5, 5, 5, 5, 5, 5, 5, 5, 5, 5]",https://www.wildberries.ru/catalog/155565431/f...,"[[Классное средство для удаления ржавчины, с п...","[[[0.14049945771694183, 0.40156668424606323, 0...","[Классное средство для удаления ржавчины, с по...","[Классное средство для удаления ржавчины, с по...","[[0.1543765664100647, 0.3817707598209381, 0.02...",0.176291,10,5.0,85,"{'negative': [], 'positive': ['Классное средст..."
220,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,"[5, 5, 5, 5, 5, 5, 5, 5]",https://www.wildberries.ru/catalog/155565431/f...,"[[Ржавчину очищает.], [Гель изо всех, что брал...","[[[0.20013216137886047, 0.289104163646698, 0.1...",[Все понравилось! Ржавчину очищает. Наношу кис...,"[Ржавчину очищает. Наношу кистью, очень удобно...","[[0.3261110484600067, 0.3369747996330261, -0.1...",0.166885,8,5.0,40,"{'negative': [], 'positive': ['удаляет ржавчин..."
221,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,"[5, 5, 5]",https://www.wildberries.ru/catalog/155565431/f...,"[[Даже старую коррозию разило, но за сутки, а ...","[[[0.2944752871990204, 0.2298327535390854, 0.8...","[Даже старую коррозию разело, но за сутки, а т...","[Даже старую коррозию разило, но за сутки, а т...","[[0.3625515401363373, 0.21077677607536316, 0.9...",0.245379,3,5.0,31,"{'negative': [], 'positive': ['Не понравилось,..."
222,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,"[5, 5, 5, 5]",https://www.wildberries.ru/catalog/155565431/f...,"[[Реально очищает от ржавчины, оставляя щербат...","[[[0.36465004086494446, 0.06613942235708237, 0...",[Как и описывалось в отзывах - это крутая штук...,"[Реально очищает от ржавчины, оставляя щербаты...","[[0.3813174068927765, 0.0376826673746109, 0.55...",0.239078,4,5.0,23,"{'negative': [], 'positive': ['Хорошо убирает ..."


,mean_distance,elem_count,review_rating_mean,word_count
count,37.000000,37.000000,37.000000,37.000000
mean,0.185329,9.000000,4.289180,66.000000
std,0.043760,8.239471,0.662741,63.644411
min,0.098123,3.000000,3.000000,17.000000
25%,0.158870,4.000000,3.750000,27.000000
50%,0.188750,7.000000,4.342857,40.000000
75%,0.215158,10.000000,5.000000,64.000000
max,0.263199,35.000000,5.000000,236.000000


'kmeans'

,category,product,review_rating,url,sentences,sentence_embeddings,review_full_text,combined_sentences,combined_sentences_embeddings,mean_distance,elem_count,review_rating_mean,word_count,key_thoughts
196,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,"[5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, ...",https://www.wildberries.ru/catalog/155565431/f...,"[[Не очищал вообще никак.], [не стекает], [Не ...","[[[0.12246482819318771, 0.38563504815101624, 0...","[В принципе, рабочая штука.\nНе очищал вообще ...",[Не очищал вообще никак. Огромный + то что кис...,"[[0.25050216913223267, 0.4493422508239746, -0....",0.129867,36,4.666667,258,{'negative': ['Сотрудники ложку утащили пасты?...
197,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,"[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",https://www.wildberries.ru/catalog/155565431/f...,"[[Брала для бытовых нужд, очистить полочку в в...","[[[-0.5432770848274231, 0.33802691102027893, 0...","[Брала для бытовых нужд,очистить полочку в ван...","[Брала для бытовых нужд, очистить полочку в ва...","[[-0.5272312760353088, 0.34916001558303833, -0...",0.285124,21,5.000000,153,"{'negative': [], 'positive': ['Отлично удаляет..."
198,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,"[1, 5, 5, 5, 3, 1, 5, 5, 3]",https://www.wildberries.ru/catalog/155565431/f...,"[[Не полная.], [Надеюсь средство также поможет...","[[[-0.24720291793346405, 0.31582504510879517, ...","[Банка вскрыта.Не полная.Жаль,что сразу не пос...","[Не полная. Жаль, что сразу не посмотрел. Напи...","[[0.4564327299594879, 0.5242350101470947, 0.28...",0.305084,9,3.666667,54,"{'negative': ['Так не о чем, эффекта почти нет..."
199,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,"[5, 5, 5, 5, 5, 5, 4, 5]",https://www.wildberries.ru/catalog/155565431/f...,"[[В деле пока не пробовала.], [Не пробовал ну ...","[[[0.4644465148448944, 0.5439950823783875, 0.8...",[Все пришло хорошо упаковано. В деле пока не п...,"[В деле пока не пробовала., Не пробовал ну на ...","[[0.48877429962158203, 0.5554664731025696, 0.8...",0.284846,8,4.875000,46,"{'negative': [], 'positive': ['В деле пока не ..."
200,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,"[5, 5, 5, 5]",https://www.wildberries.ru/catalog/155565431/f...,[[Отличное средство для чистки в быту многих п...,"[[[0.28691887855529785, 0.05522089824080467, 0...",[Отличное средство для чистки в быту многих по...,[Отличное средство для чистки в быту многих по...,"[[0.3062698543071747, 0.03441806510090828, 0.5...",0.347131,4,5.000000,27,"{'negative': [], 'positive': ['Разъедает ржавч..."


,mean_distance,elem_count,review_rating_mean,word_count
count,36.000000,36.000000,36.000000,36.000000
mean,0.270476,10.694444,4.424240,77.944444
std,0.092022,9.462164,0.616886,69.903176
min,0.123740,3.000000,3.000000,15.000000
25%,0.192040,4.000000,3.950000,26.750000
50%,0.276375,6.000000,4.611111,47.000000
75%,0.332517,15.250000,5.000000,121.750000
max,0.491639,36.000000,5.000000,258.000000


In [21]:
final_df_hierarchical.iloc[3]["combined_sentences"]

['Брал для поездки на определенное озеро , крутой подъем с рыхлым песком , но в моем случае не сработало думаю на горизонтальной дороге работать будет Брал для поездки на определенное озеро , крутой подъем с рыхлым песком , но в моем случае не сработало думаю на горизонтальной дороге работать будет',
 'Брали на мазда с 5, теперь застрять на даче не страшно!!! Брали на мазда с 5, теперь застрять на даче не страшно!!!',
 'Я конечно не засаживал машину по самые яйца , но оказывался в таких ситуациях когда находился один или с маленьким ребёнком в машине и и для таких моментов - этот деваем самый настоящий друг . Про Надёжность скажу так опыта с другими девайсами работы не было но про этот если аккуратно использовать то прослужит долго . Я конечно не засаживал машину по самые яйца , но оказывался в таких ситуациях когда находился один или с маленьким ребёнком в машине и и для таких моментов - этот деваем самый настоящий друг . Про Надёжность скажу так опыта с другими девайсами работы не бы

In [22]:
final_df_hierarchical.iloc[3]["key_thoughts"]["positive"]

['пока не устроился в очень глубокий снег(дверь еле открыл )',
 'Задний привод, выехать не мог, одел цепи, выехал бодро.',
 'По красоте хорошие ещё не обкатал',
 'Хорошая вещь! Уже несколько раз выручали этой зимой. Для шин 185 то же подходят. Главное научиться правильно закреплять, что бы не бегать по району искать. Читайте инструкцию. Спасибо производителям!']

In [23]:
final_df_hierarchical.iloc[3]["key_thoughts"]["negative"]

['Не удобные, застежка ремня сделана не удобно, если удастся одеть, то из снега вытягивают',
 'Внешне вроде неплохой, но швы не закреплены, замок один согнутый пришел🥺, обидно так ждала',
 'Не смогла выехать с парковки после снегопада, одела цепи, колёса с цепями вырыли яму, с места машина не сдвинулась.',
 'Не удобные, застежка ремня сделана не удобно, если удастся одеть, то из снега вытягивают']

# sentences

In [24]:
import numpy as np
import pandas as pd
from hdbscan import HDBSCAN
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_distances
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist
from tqdm import tqdm
from loguru import logger

logger.add('./reviews_keywords/clustering.log', encoding='utf-8')
# Перевод модели в режим FP16, если это возможно
if torch.cuda.is_available():
    model_classification = model_classification.half()

def find_centroid(embeddings):
    return np.mean(embeddings, axis=0)

def extract_key_thought(sentences, embeddings = None):
    centroid = find_centroid(embeddings)
    similarities = cosine_similarity(embeddings, [centroid])
    key_sentence_index = np.argmax(similarities)
    return sentences[key_sentence_index]

def count_words(cluster_sentences):
    words = " ".join(cluster_sentences).split()
    return len(words)

def compute_mean_distance(cluster_indices, embeddings, metric='cosine'):
    if len(cluster_indices) < 2:
        return 0
    cluster_embeddings = embeddings[cluster_indices]
    pairwise_distances = pdist(cluster_embeddings, metric=metric)
    return np.mean(pairwise_distances)

def aggregate_clusters(group, labels, embeddings):
    group['final_cluster'] = labels
    group['mean_distance'] = group['final_cluster'].apply(
        lambda cluster_id: compute_mean_distance(
            np.where(labels == cluster_id)[0], embeddings)
    )
    
    grouped_clusters = group.groupby('final_cluster').agg({
        'category': 'first',
        'product': 'first',
        'review_rating': 'mean',
        'url': 'first',
        'sentences': list,
        'sentence_embeddings': list,
        'mean_distance': 'first',
    }).reset_index(drop=True)
    
    grouped_clusters['elem_count'] = grouped_clusters['sentences'].apply(len)
    grouped_clusters['word_count'] = grouped_clusters['sentences'].apply(lambda x: sum([len(" ".join(s).split()) for s in x]))
    grouped_clusters['key_thought'] = grouped_clusters.apply(
        lambda row: extract_key_thought(row['sentences'], row['sentence_embeddings']), axis=1)
    
    return grouped_clusters.sort_values(by='word_count', ascending=False)

def cluster_with_hdbscan(group, min_cluster_size=3, min_samples=3):
    embeddings = np.vstack(group["sentence_embeddings"].values)
    distance_matrix = cosine_distances(embeddings)
    
    hdb = HDBSCAN(metric='precomputed', min_cluster_size=min_cluster_size, min_samples=min_samples)
    labels = hdb.fit_predict(distance_matrix)
    
    return aggregate_clusters(group, labels, embeddings)

def cluster_with_hierarchical(group, max_distance=0.5):
    embeddings = np.array(list(group["sentence_embeddings"]))
    distance_matrix = pdist(embeddings, metric='cosine')
    linkage_matrix = linkage(distance_matrix, method='ward')
    labels = fcluster(linkage_matrix, t=max_distance, criterion='distance')
    
    return aggregate_clusters(group, labels, embeddings)

def cluster_with_kmeans(group, n_clusters=5):
    embeddings = np.array(list(group["sentence_embeddings"]))
    n_samples = embeddings.shape[0]
    
    if n_samples < n_clusters:
        n_clusters = max(2, n_samples)
    
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    labels = kmeans.fit_predict(embeddings)
    
    return aggregate_clusters(group, labels, embeddings)

def cluster_product_reviews(df, clustering_method, **kwargs):
    final_clusters_list = []
    grouped = df.groupby('product')

    for product, group in tqdm(grouped, desc="Processing products"):
        clustered_group = clustering_method(group, **kwargs)
        final_clusters_list.append(clustered_group)
    
    return pd.concat(final_clusters_list, ignore_index=True)

# Пример использования
final_df_hdbscan = cluster_product_reviews(df_filtered_embed, cluster_with_hdbscan, min_cluster_size=3, min_samples=3)
final_df_hierarchical = cluster_product_reviews(df_filtered_embed, cluster_with_hierarchical, max_distance=0.3)
final_df_kmeans = cluster_product_reviews(df_filtered_embed, cluster_with_kmeans, n_clusters=20)

# Сортировка для вывода
final_df_hdbscan = final_df_hdbscan.sort_values("word_count", ascending=False)
final_df_hierarchical = final_df_hierarchical.sort_values("word_count", ascending=False)
final_df_kmeans = final_df_kmeans.sort_values("word_count", ascending=False)

# Вывод результатов
display(f"hdbscan")
display(final_df_hdbscan[final_df_hdbscan["elem_count"] > 2].sort_values("product", ascending=False).head(5))
display(final_df_hdbscan[final_df_hdbscan["elem_count"] > 2].describe())
display(f"hierarchical")
display(final_df_hierarchical[final_df_hierarchical["elem_count"] > 2].sort_values("product", ascending=False).head(5))
display(final_df_hierarchical[final_df_hierarchical["elem_count"] > 2].describe())
display(f"kmeans")
display(final_df_kmeans[final_df_kmeans["elem_count"] > 2].sort_values("product", ascending=False).head(5))
display(final_df_kmeans[final_df_kmeans["elem_count"] > 2].describe())

Processing products: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:07<00:00,  2.26it/s]


'hdbscan'

,category,product,review_rating,url,sentences,sentence_embeddings,mean_distance,elem_count,word_count,key_thought
27,/Автотовары/OFFroad,Фрегат Лифт Подвеска / Лифт комплект рессоры К...,4.500000,https://www.wildberries.ru/catalog/111627554/f...,[Встали как родные устанавливал на петро пикап...,"[[0.04173365607857704, 0.2767508924007416, -0....",0.474746,4,178,Встали как родные устанавливал на петро пикапа
24,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,4.450867,https://www.wildberries.ru/catalog/155565431/f...,"[Дождь пошёл покрылась, что было образуется пл...","[[-1.0747172832489014, 0.12837457656860352, -0...",0.559831,173,5117,Совсем не убирает ржавчину с машины.
25,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,4.868421,https://www.wildberries.ru/catalog/155565431/f...,[всю особенность подождать сутки там желательн...,"[[-0.2752877175807953, 0.27684488892555237, 0....",0.197835,38,1835,"30 минут Водой не промывал, протер тряпкой."
26,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,5.000000,https://www.wildberries.ru/catalog/155565431/f...,"[Ржавчину убирает отлично., Огонь ржавчину уби...","[[-0.095274418592453, 0.08963502943515778, 0.5...",0.217153,4,94,Ржавчину убирает отлично.
21,/Автотовары/Автокосметика и автохимия,Пахнет и Точка / Ароматизатор в машину автопар...,3.565789,https://www.wildberries.ru/catalog/197410221/f...,"[Во - первых пришел совсем другой аромат ,, во...","[[-0.47253894805908203, -0.18203142285346985, ...",0.609218,152,3879,что мой чувствительный к запахам нос просто в ...


,review_rating,mean_distance,elem_count,word_count
count,28.000000,28.000000,28.000000,28.000000
mean,4.286174,0.427073,49.035714,1558.178571
std,0.544489,0.190893,59.384174,1823.804206
min,3.000000,0.000000,3.000000,60.000000
25%,3.986842,0.288148,4.750000,165.750000
50%,4.290030,0.476154,18.500000,701.000000
75%,4.650000,0.552542,78.750000,2454.750000
max,5.000000,0.774385,173.000000,5255.000000


'hierarchical'

,category,product,review_rating,url,sentences,sentence_embeddings,mean_distance,elem_count,word_count,key_thought
751,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,4.642857,https://www.wildberries.ru/catalog/155565431/f...,[всю особенность подождать сутки там желательн...,"[[-0.2752877175807953, 0.27684488892555237, 0....",0.174323,14,765,Если рыхлая - 2-3 раза нужно наносить.
752,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,4.600000,https://www.wildberries.ru/catalog/155565431/f...,"[на следующий день высохший раствор стёр, Обра...","[[-0.16233563423156738, 0.19301316142082214, -...",0.176208,10,448,"На унитазе ржавчину убрало на раз два, чем до ..."
753,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,5.000000,https://www.wildberries.ru/catalog/155565431/f...,[Покупала для подготовки мангала перед покраск...,"[[-0.16269774734973907, 0.39143142104148865, 0...",0.185798,6,378,"Классное средство для удаления ржавчины, с пом..."
754,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,5.000000,https://www.wildberries.ru/catalog/155565431/f...,[Нанес для пробы на первый попавшийся кусок же...,"[[0.4127022922039032, 0.07070054113864899, -0....",0.244446,4,241,"Ржавчина уже хорошо въелась, пришлось нескольк..."
755,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,5.000000,https://www.wildberries.ru/catalog/155565431/f...,"[Удалял ""жучки"" на дверях авто., С крыла ржавч...","[[0.06537044048309326, 0.5978235602378845, -0....",0.150612,6,238,Отчистил этим средством всю ржавчину.


,review_rating,mean_distance,elem_count,word_count
count,102.000000,102.000000,102.000000,102.000000
mean,4.348131,0.205412,4.666667,191.598039
std,0.669810,0.048450,3.153917,180.483468
min,2.666667,0.000000,3.000000,23.000000
25%,4.000000,0.176240,3.000000,97.500000
50%,4.621429,0.212294,4.000000,146.000000
75%,5.000000,0.238630,5.000000,208.750000
max,5.000000,0.278779,26.000000,1368.000000


'kmeans'

,category,product,review_rating,url,sentences,sentence_embeddings,mean_distance,elem_count,word_count,key_thought
226,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,4.393939,https://www.wildberries.ru/catalog/155565431/f...,[всю особенность подождать сутки там желательн...,"[[-0.2752877175807953, 0.27684488892555237, 0....",0.239199,33,1512,"30 минут Водой не промывал, протер тряпкой."
227,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,4.458333,https://www.wildberries.ru/catalog/155565431/f...,[После того как промыл по инструкции на банке....,"[[0.33993497490882874, 0.18883438408374786, 0....",0.333461,24,1008,Отчистил этим средством всю ржавчину.
228,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,4.578947,https://www.wildberries.ru/catalog/155565431/f...,"[наносить кисточкой ,, Покупала для подготовки...","[[-0.3121572732925415, 0.19685323536396027, 0....",0.294562,19,763,"Классное средство для удаления ржавчины, с пом..."
229,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,4.750000,https://www.wildberries.ru/catalog/155565431/f...,"[после высыхания остаётся, Очень хорошее средс...","[[0.3377434015274048, -0.0954262763261795, -0....",0.376692,20,727,Отлично удаляет даже ооо очень заставляют ржав...
230,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,4.866667,https://www.wildberries.ru/catalog/155565431/f...,"[подтёки на крашеных элементах убирает, Убирае...","[[0.17964382469654083, 0.1780298352241516, 0.0...",0.358767,15,501,Ржавчину убирает отлично.


,review_rating,mean_distance,elem_count,word_count
count,121.000000,121.000000,121.000000,121.000000
mean,4.242003,0.417190,10.024793,324.404959
std,0.706409,0.115725,8.608584,389.504762
min,1.333333,0.000000,3.000000,31.000000
25%,3.833333,0.357025,4.000000,107.000000
50%,4.333333,0.418157,7.000000,202.000000
75%,4.933333,0.502032,12.000000,327.000000
max,5.000000,0.617137,46.000000,2179.000000


# Обработка результатов

In [28]:
import numpy as np
import pandas as pd
from hdbscan import HDBSCAN
from sklearn.metrics.pairwise import cosine_distances
from tqdm import tqdm

def compute_mean_distance(cluster_indices, embeddings):
    if len(cluster_indices) < 2:
        return 0
    cluster_embeddings = embeddings[cluster_indices]
    pairwise_distances = pdist(cluster_embeddings, metric='cosine')
    return np.mean(pairwise_distances)

def aggregate_embeddings(group):
    # Агрегируем эмбеддинги для предложений одного отзыва
    return np.mean(np.vstack(group["sentence_embeddings"]), axis=0)

def cluster_product_reviews_with_hdbscan(df, min_cluster_size=3, min_samples=3):
    final_clusters_list = []

    grouped = df.groupby('product')

    for product, group in tqdm(grouped, desc="Processing products"):
        embeddings = np.vstack(group["sentence_embeddings"].values)
        
        # Вычисляем косинусное расстояние
        distance_matrix = cosine_distances(embeddings)
        
        # Применяем HDBSCAN
        hdb = HDBSCAN(metric='precomputed', min_cluster_size=min_cluster_size, min_samples=min_samples)
        labels = hdb.fit_predict(distance_matrix)
        
        # Добавляем метки кластеров в DataFrame
        group['final_cluster'] = labels
        
        group['mean_distance'] = group['final_cluster'].apply(
            lambda cluster_id: compute_mean_distance(
                np.where(labels == cluster_id)[0], embeddings)
        )
        
        grouped_clusters = group.groupby('final_cluster').agg({
            'category': 'first',
            'product': 'first',
            'review_rating': 'mean',
            'url': 'first',
            'sentences': list,
            'sentence_embeddings': list,
            'mean_distance': 'first',
        }).reset_index(drop=True)
        
        grouped_clusters['elem_count'] = grouped_clusters['sentences'].apply(len)  
        grouped_clusters['word_count'] = grouped_clusters['sentences'].apply(lambda x: sum([len(" ".join(s).split()) for s in x]))        
        grouped_clusters['key_thought'] = grouped_clusters.apply(
            lambda row: extract_key_thought(row['sentences'], row['sentence_embeddings']), axis=1)
        
        grouped_clusters = grouped_clusters.sort_values(by='word_count', ascending=False)
        final_clusters_list.append(grouped_clusters)
    
    return pd.concat(final_clusters_list, ignore_index=True)

# Применяем кластеризацию ко всему DataFrame
min_cluster_size = 3  # Минимальный размер кластера
min_samples = 3       # Минимальное количество точек в кластере

final_df = cluster_product_reviews_with_hdbscan(df_filtered_reviews, min_cluster_size, min_samples)
final_df = final_df.sort_values("word_count", ascending=False)
final_df

Processing products: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:05<00:00,  2.68it/s]


,category,product,review_rating,url,sentences,sentence_embeddings,mean_distance,elem_count,word_count,key_thought
13,/Автотовары/OFFroad,ВПМ / Антибукс - антипробуксовочные траки утол...,4.316901,https://www.wildberries.ru/catalog/48839109/fe...,"[Можно было отлив снизу сделать, чтобы не выпа...","[[-0.2880934774875641, 0.06186602637171745, 0....",0.368512,142,5255,"2 раза спасали на гололедице, шлифовал на мест..."
24,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,4.450867,https://www.wildberries.ru/catalog/155565431/f...,"[Дождь пошёл покрылась, что было образуется пл...","[[-1.0747172832489014, 0.12837457656860352, -0...",0.559831,173,5117,Совсем не убирает ржавчину с машины.
19,/Автотовары/OFFroad,ПК ЛИМ / Браслеты цепи противоскольжения,4.251572,https://www.wildberries.ru/catalog/143132420/f...,"[Покупал набор из шести штук, но хватает 4 (на...","[[0.09786882996559143, 0.07219353318214417, 0....",0.506770,159,4945,"Не удобные, застежка ремня сделана не удобно, ..."
20,/Автотовары/OFFroad,ПК ЛИМ / Цепи противоскольжения для легковых а...,4.251572,https://www.wildberries.ru/catalog/143132161/f...,"[Покупал набор из шести штук, но хватает 4 (на...","[[0.06749556213617325, 0.034723978489637375, 0...",0.506064,159,4945,"Не удобные, застежка ремня сделана не удобно, ..."
21,/Автотовары/Автокосметика и автохимия,Пахнет и Точка / Ароматизатор в машину автопар...,3.565789,https://www.wildberries.ru/catalog/197410221/f...,"[Во - первых пришел совсем другой аромат ,, во...","[[-0.47253894805908203, -0.18203142285346985, ...",0.609218,152,3879,что мой чувствительный к запахам нос просто в ...
16,/Автотовары/OFFroad,ВЫРУЧАЙКА / Антибукс Противобуксовочные траки ...,4.235772,https://www.wildberries.ru/catalog/62827233/fe...,"[В снегу действительно помогло., Не с первого ...","[[-0.19982746243476868, 0.1130455955862999, 0....",0.546376,123,3876,"Материал качество нормальное, использовать нуж..."
1,/Автотовары/OFFroad,Hangkai / Лебедка электрическая влагозащитная ...,4.561644,https://www.wildberries.ru/catalog/124191551/f...,"[Внутри лучше разобрать и смазать, так как сма...","[[0.07086107134819031, 0.4617305099964142, -0....",0.477561,73,2625,Перестала тянуть трос после двух использовании
14,/Автотовары/OFFroad,ВПМ / Антибукс - антипробуксовочные траки утол...,4.354167,https://www.wildberries.ru/catalog/48839109/fe...,"[Зацеп между собой не продуманный., Не во всех...","[[0.6099634170532227, 0.42926353216171265, -0....",0.647027,96,2398,Зацеп между собой не продуманный.
25,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,4.868421,https://www.wildberries.ru/catalog/155565431/f...,[всю особенность подождать сутки там желательн...,"[[-0.2752877175807953, 0.27684488892555237, 0....",0.197835,38,1835,"30 минут Водой не промывал, протер тряпкой."
22,/Автотовары/Автокосметика и автохимия,Пахнет и Точка / Ароматизатор в машину автопар...,3.947368,https://www.wildberries.ru/catalog/197410221/f...,"[если нос не приложить к ароматизатору …, На з...","[[0.06371931731700897, 0.32030782103538513, 0....",0.305465,38,1650,"Давно искала аромат для машины, чтобы пахло на..."


In [29]:
import numpy as np
import pandas as pd
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram
from scipy.spatial.distance import pdist
from tqdm import tqdm
import matplotlib.pyplot as plt

def compute_mean_distance(cluster_indices, embeddings):
    if len(cluster_indices) < 2:
        return 0
    cluster_embeddings = embeddings[cluster_indices]
    pairwise_distances = pdist(cluster_embeddings, metric='cosine')
    return np.mean(pairwise_distances)

def cluster_product_reviews(df, max_distance=0.5):
    grouped = df.groupby('product')
    final_clusters_list = []

    for product, group in tqdm(grouped, desc="Processing products"):
        sentences = group["sentences"].tolist()
        embeddings = np.array(list(group["sentence_embeddings"]))
        
        distance_matrix = pdist(embeddings, metric='cosine')
        linkage_matrix = linkage(distance_matrix, method='ward')
        initial_clusters = fcluster(linkage_matrix, t=max_distance, criterion='distance')
        
        group['final_cluster'] = initial_clusters
        group['mean_distance'] = group['final_cluster'].apply(
            lambda cluster_id: compute_mean_distance(
                np.where(initial_clusters == cluster_id)[0], embeddings)
        )
        
        grouped_clusters = group.groupby('final_cluster').agg({
            'category': 'first',
            'product': 'first',
            'review_rating': 'mean',
            'url': 'first',
            'sentences': list,
            'sentence_embeddings': list,
            'mean_distance': 'first',
        }).reset_index(drop=True)
        
        grouped_clusters['elem_count'] = grouped_clusters['sentences'].apply(len)  
        grouped_clusters['word_count'] = grouped_clusters['sentences'].apply(count_words)        
        grouped_clusters['key_thought'] = grouped_clusters.apply(
            lambda row: extract_key_thought(row['sentences'], row['sentence_embeddings']), axis=1)
        
        grouped_clusters = grouped_clusters.sort_values(by='word_count', ascending=False)
        final_clusters_list.append(grouped_clusters)
    
    return pd.concat(final_clusters_list, ignore_index=True)

# Применяем кластеризацию ко всему DataFrame
final_df = cluster_product_reviews(df_filtered_reviews, max_distance=0.3)
# Для вывода финального результата
final_df = final_df.sort_values("word_count", ascending=False)
display(final_df.info())
final_df[final_df["elem_count"] > 2].sort_values("product", ascending=False)

Processing products: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:01<00:00,  8.13it/s]

<class 'cudf.core.dataframe.DataFrame'>
Index: 881 entries, 173 to 61
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   category             881 non-null    object
 1   product              881 non-null    object
 2   review_rating        881 non-null    float64
 3   url                  881 non-null    object
 4   sentences            881 non-null    list
 5   sentence_embeddings  881 non-null    list
 6   mean_distance        881 non-null    float64
 7   elem_count           881 non-null    int64
 8   word_count           881 non-null    int64
 9   key_thought          881 non-null    object
dtypes: float64(2), int64(2), list(2), object(4)
memory usage: 11.0+ MB


None

,category,product,review_rating,url,sentences,sentence_embeddings,mean_distance,elem_count,word_count,key_thought
751,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,4.642857,https://www.wildberries.ru/catalog/155565431/f...,[всю особенность подождать сутки там желательн...,"[[-0.2752877175807953, 0.27684488892555237, 0....",0.174323,14,133,Если рыхлая - 2-3 раза нужно наносить.
752,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,4.600000,https://www.wildberries.ru/catalog/155565431/f...,"[на следующий день высохший раствор стёр, Обра...","[[-0.16233563423156738, 0.19301316142082214, -...",0.176208,10,87,"На унитазе ржавчину убрало на раз два, чем до ..."
753,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,5.000000,https://www.wildberries.ru/catalog/155565431/f...,[Покупала для подготовки мангала перед покраск...,"[[-0.16269774734973907, 0.39143142104148865, 0...",0.185798,6,62,"Классное средство для удаления ржавчины, с пом..."
754,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,5.000000,https://www.wildberries.ru/catalog/155565431/f...,[Нанес для пробы на первый попавшийся кусок же...,"[[0.4127022922039032, 0.07070054113864899, -0....",0.244446,4,45,"Ржавчина уже хорошо въелась, пришлось нескольк..."
755,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,5.000000,https://www.wildberries.ru/catalog/155565431/f...,"[Удалял ""жучки"" на дверях авто., С крыла ржавч...","[[0.06537044048309326, 0.5978235602378845, -0....",0.150612,6,40,Отчистил этим средством всю ржавчину.
...,...,...,...,...,...,...,...,...,...,...
4,/Автотовары/OFFroad,Hangkai / Лебедка электрическая влагозащитная ...,5.000000,https://www.wildberries.ru/catalog/124191551/f...,"[Лебедка хорошая лучика тянет на ура)👍👍👍, Тяне...","[[-0.2121036797761917, 0.01784498244524002, 0....",0.243116,4,32,Лебедка хорошая лучика тянет на ура)👍👍👍
5,/Автотовары/OFFroad,Hangkai / Лебедка электрическая влагозащитная ...,5.000000,https://www.wildberries.ru/catalog/124191551/f...,"[Внутри лучше разобрать и смазать, так как сма...","[[0.07086107134819031, 0.4617305099964142, -0....",0.179995,4,29,"Внутри лучше разобрать и смазать, так как смаз..."
6,/Автотовары/OFFroad,Hangkai / Лебедка электрическая влагозащитная ...,5.000000,https://www.wildberries.ru/catalog/124191551/f...,"[то влаги в ней не будет ,, водой сверху защит...","[[-0.10012279450893402, 0.5909180045127869, 0....",0.231133,3,21,"то влаги в ней не будет ,"
8,/Автотовары/OFFroad,Hangkai / Лебедка электрическая влагозащитная ...,4.666667,https://www.wildberries.ru/catalog/124191551/f...,"[Только инструкции нет, Рычаг свободного хода ...","[[0.46362248063087463, 0.7040594816207886, -0....",0.245024,3,18,Вот только ремешка нет на крюк в комплекте


In [30]:
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_distances
from tqdm import tqdm

def compute_mean_distance(cluster_indices, embeddings):
    if len(cluster_indices) < 2:
        return 0
    cluster_embeddings = embeddings[cluster_indices]
    pairwise_distances = cosine_distances(cluster_embeddings)
    return np.mean(pairwise_distances)

def cluster_product_reviews_with_kmeans(df, n_clusters=5):
    grouped = df.groupby('product')
    final_clusters_list = []

    for product, group in tqdm(grouped, desc="Processing products"):
        embeddings = np.array(list(group["sentence_embeddings"]))
        n_samples = embeddings.shape[0]
        
        if n_samples < n_clusters:
            n_clusters = max(2, n_samples)  # Корректируем количество кластеров, если данных меньше
        
        # Применяем K-means
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        labels = kmeans.fit_predict(embeddings)
        
        group['final_cluster'] = labels
        
        # Вычисляем среднее расстояние для каждого кластера
        group['mean_distance'] = group['final_cluster'].apply(
            lambda cluster_id: compute_mean_distance(
                np.where(labels == cluster_id)[0], embeddings)
        )
        
        grouped_clusters = group.groupby('final_cluster').agg({
            'category': 'first',
            'product': 'first',
            'review_rating': 'mean',
            'url': 'first',
            'sentences': list,
            'sentence_embeddings': list,
            'mean_distance': 'first',  # Добавляем среднее расстояние в итоговый DataFrame
        }).reset_index(drop=True)
        
        grouped_clusters['elem_count'] = grouped_clusters['sentences'].apply(len)
        grouped_clusters['word_count'] = grouped_clusters['sentences'].apply(count_words)
        grouped_clusters['key_thought'] = grouped_clusters.apply(
            lambda row: extract_key_thought(row['sentences'], row['sentence_embeddings']), axis=1)
        
        grouped_clusters = grouped_clusters.sort_values(by='word_count', ascending=False)
        final_clusters_list.append(grouped_clusters)
    
    return pd.concat(final_clusters_list, ignore_index=True)

# Применяем кластеризацию ко всему DataFrame
n_clusters = 20  # Задаем количество кластеров
final_df = cluster_product_reviews_with_kmeans(df_filtered_reviews, n_clusters=n_clusters)
final_df = final_df.sort_values("word_count", ascending=False)
final_df[final_df["elem_count"] > 2].sort_values("product", ascending=False)

Processing products: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:07<00:00,  2.00it/s]


,category,product,review_rating,url,sentences,sentence_embeddings,mean_distance,elem_count,word_count,key_thought
42,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,4.525773,https://www.wildberries.ru/catalog/155565431/f...,[всю особенность подождать сутки там желательн...,"[[-0.2752877175807953, 0.27684488892555237, 0....",0.346020,97,658,"30 минут Водой не промывал, протер тряпкой."
43,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,4.642857,https://www.wildberries.ru/catalog/155565431/f...,"[Дождь пошёл покрылась, что было образуется пл...","[[-1.0747172832489014, 0.12837457656860352, -0...",0.454276,70,429,"удаляет ржавчину на глазах до металла ,"
44,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,4.395833,https://www.wildberries.ru/catalog/155565431/f...,"[на проблемные места, за час !, Полная хрень.,...","[[0.011556755751371384, -0.09506919234991074, ...",0.656352,48,238,"если пополнить на кп при обработке ,"
39,/Автотовары/Автокосметика и автохимия,Пахнет и Точка / Ароматизатор в машину автопар...,3.893617,https://www.wildberries.ru/catalog/197410221/f...,"[если нос не приложить к ароматизатору …, На з...","[[0.06371931731700897, 0.32030782103538513, 0....",0.404765,94,634,"Давно искала аромат для машины, чтобы пахло на..."
40,/Автотовары/Автокосметика и автохимия,Пахнет и Точка / Ароматизатор в машину автопар...,3.602941,https://www.wildberries.ru/catalog/197410221/f...,"[Во - первых пришел совсем другой аромат ,, ОЦ...","[[-0.47253894805908203, -0.18203142285346985, ...",0.580406,68,384,"Купила авто парфюм, запах восхитительный 🔥"
41,/Автотовары/Автокосметика и автохимия,Пахнет и Точка / Ароматизатор в машину автопар...,3.096774,https://www.wildberries.ru/catalog/197410221/f...,"[во вторых исчез, за 5 дней полностью ,, в тре...","[[-0.505354106426239, 0.1778121143579483, -0.5...",0.641620,31,110,Он даже не пахнет.
36,/Автотовары/OFFroad,ПК ЛИМ / Цепи противоскольжения для легковых а...,4.128571,https://www.wildberries.ru/catalog/143132161/f...,"[Маленькие карабины не он., будет заправлять р...","[[0.2422276884317398, 0.20851214230060577, -0....",0.306248,70,518,"Не удобные, застежка ремня сделана не удобно, ..."
37,/Автотовары/OFFroad,ПК ЛИМ / Цепи противоскольжения для легковых а...,4.372549,https://www.wildberries.ru/catalog/143132161/f...,"[Удобные, но постепенно привыкаешь., как на фо...","[[0.08902470767498016, 0.07573284208774567, 0....",0.489963,51,294,Напишу позже пака не пробовали
38,/Автотовары/OFFroad,ПК ЛИМ / Цепи противоскольжения для легковых а...,4.315789,https://www.wildberries.ru/catalog/143132161/f...,"[Покупал набор из шести штук, но хватает 4 (на...","[[0.06749556213617325, 0.034723978489637375, 0...",0.629828,38,177,"Отличные браслеты, установил на колеса без про..."
33,/Автотовары/OFFroad,ПК ЛИМ / Браслеты цепи противоскольжения,4.142857,https://www.wildberries.ru/catalog/143132420/f...,"[Маленькие карабины не он., будет заправлять р...","[[0.23492081463336945, 0.20205345749855042, -0...",0.338690,91,670,"Не удобные, застежка ремня сделана не удобно, ..."
